In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.colors import LogNorm, Normalize
from matplotlib.ticker import (
    FixedLocator,
    MultipleLocator,
    FuncFormatter,
    StrMethodFormatter,
)
from matplotlib.lines import Line2D
from matplotlib.offsetbox import AnchoredText
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np

from covvvr import CVIntegrator
from covvvr.functions import *

from scripts.constants import DATA_DIR


---

# Variance Percentage Reduction for 2 Control Variates 

## Constants

In [ ]:
func = "16d_gaussian"
iters = 50
evts = 5000
N = 100

file_path = DATA_DIR / f"2cv_{func}_{iters}x{evts}_avg{N}.npz"


In [ ]:
# Ratio of main plot to the side plots
ratio = [11, 2]
## Fontsizes of
# the figure titles
title_fs = 26
# the colorbar tick numbers
cbar_tick_fs = 15
# the colorbar label
cbar_label_fs = 20
# the axis label
axis_label_fs = 20
# the axis tick numbers
axis_tick_fs = 17

# Linewidth
lw = 3
# Colormap for the colorbar/heatmap
cmap = "RdPu"
# Colors to use for the variance and correlation coefficient lines
line_cmap = "magma"
colors = {
    "corrcoefs": colormaps[line_cmap](0.3),
    "variances": colormaps[line_cmap](0.7),
}
# Use a log colorbar?
do_log = False
# Share colorbar?
share_bar = False
# For side axes from 0 to 1
major_intr = 0.5
minor_intr = 0.25
major_locator = np.arange(0, 1 + major_intr, major_intr)
minor_locator = np.arange(0, 1 + minor_intr, minor_intr)
# For side axes from 1 to iters - 1
num_ticks = 5
iters_major_locator = np.concatenate(
    ([1], np.arange(0, iters, iters // num_ticks)[1:], [iters - 1])
)


## Plot Heatmap

In [ ]:
def make_plot(fig, func, ax, variances, corrcoefs, perc_diff, vmax=None):
    vars_norm = variances / np.max(variances)

    # Plot the Variances
    ax[0, 0].plot(
        vars_norm,
        range(1, iters),
        lw=lw,
        c=colors["variances"],
        label="Normalized\nVariance",
    )
    # Plot the correlation coefficients
    ax[0, 0].plot(
        corrcoefs,
        range(1, iters),
        lw=lw,
        c=colors["corrcoefs"],
        label="Correlation\nCoefficient",
    )
    ax[0, 0].set_xlim(-0.05, 1.05)
    ax[0, 0].set_ylim(1, iters - 1)
    ax[0, 0].set_ylabel("Control Variate Iteration", fontsize=axis_label_fs)
    # Make sure all the ticks are looking good
    ax[0, 0].yaxis.set_major_locator(FixedLocator(iters_major_locator))
    ax[0, 0].xaxis.set_minor_locator(FixedLocator(minor_locator))
    ax[0, 0].xaxis.set_major_locator(FixedLocator(major_locator))
    # No decimal points for intergers
    ax[0, 0].xaxis.set_major_formatter(
        FuncFormatter(lambda s, pos: f"{s:.0f}" if s == int(s) else f"{s:.1f}")
    )
    ax[0, 0].xaxis.set_minor_locator(FixedLocator(minor_locator))
    ax[0, 0].tick_params(axis="both", which="major", labelsize=axis_tick_fs)

    # Either use a log colorbar or start the colorbar at zero
    kwargs = {"norm": LogNorm(vmax=vmax)} if do_log else {"vmin": 0, "vmax": vmax}
    h = ax[0, 1].imshow(perc_diff, cmap=cmap, **kwargs)
    ax[0, 1].invert_yaxis()
    ax[0, 1].tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
    # Create legend for axis plots but put it in heatmap
    handles = [
        Line2D([0], [0], color=colors["variances"], lw=4, label="Normalized\nVariance"),
        Line2D(
            [0], [0], color=colors["corrcoefs"], lw=4, label="Correlation\nCoefficient"
        ),
    ]
    ax[0, 1].legend(
        handles=handles, loc="upper left", fontsize=15, fancybox=True, shadow=True
    )
    # Create title name, i.e. function name, put it in the plot
    funcbox = AnchoredText(
        func.replace("_", " ").title(),
        frameon=False,
        loc="upper right",
        prop=dict(fontsize=title_fs, fontweight="bold", color="#000000"),
    )
    ax[0, 1].add_artist(funcbox)

    ax[1, 0].set_visible(False)

    # Same as with ax[0, 0] but the axes are flipped
    ax[1, 1].plot(range(1, iters), vars_norm, lw=lw, c=colors["variances"])
    ax[1, 1].plot(range(1, iters), corrcoefs, lw=lw, c=colors["corrcoefs"])
    ax[1, 1].set_ylim(-0.05, 1.05)
    ax[1, 1].set_xlim(1, iters - 1)
    ax[1, 1].set_xlabel("Control Variate Iteration", fontsize=axis_label_fs)
    ax[1, 1].xaxis.set_major_locator(FixedLocator(iters_major_locator))
    ax[1, 1].yaxis.set_minor_locator(FixedLocator(minor_locator))
    ax[1, 1].yaxis.set_major_locator(FixedLocator(major_locator))
    # Remove the '1' on the axis because it overlaps with the 1 from ax[0, 0]
    ax[1, 1].yaxis.set_major_locator(FixedLocator(major_locator[:-1]))
    ax[1, 1].yaxis.set_major_formatter(
        FuncFormatter(lambda s, pos: f"{s:.0f}" if s == int(s) else f"{s:.1f}")
    )
    ax[1, 1].yaxis.set_minor_locator(FixedLocator(minor_locator))
    ax[1, 1].tick_params(axis="both", which="major", labelsize=axis_tick_fs)

    # Colorbar
    if not share_bar:
        axins = ax[0, 1].inset_axes(bounds=[-0.18, 1.07, 1.18, 0.1])
        cbar = fig.colorbar(
            h,
            cax=axins,
            orientation="horizontal",
            format=StrMethodFormatter("{x:>5.1f}%"),
        )
        cbar.ax.tick_params(labelsize=cbar_tick_fs)
        cbar.ax.xaxis.set_ticks_position("top")
        cbar.ax.xaxis.set_label_position("top")
        cbar.set_label(
            "Variance Percentage Reduction", labelpad=20, fontsize=cbar_label_fs
        )

    # Squish 'em together
    plt.subplots_adjust(wspace=0, hspace=0)
    return h, ax[0, 1]


In [ ]:
funcs = ["16d_gaussian", "scalar_top_loop"]

# Create figure to hold multiple subfigures
fig = plt.figure(figsize=(20, 10))
subfigs = fig.subfigures(1, len(funcs))

max_vrp = 0
datas = []
for func in funcs:
    file_path = DATA_DIR / f"2cv_{func}_{iters}x{evts}_avg{N}.npz"
    data = np.load(file_path)
    datas.append(data)

    # Find maximum VRP to use as limit on shared colorbar
    if share_bar:
        func_max_vrp = 100 * np.max(data["perc_diff"])
        if func_max_vrp > max_vrp:
            max_vrp = func_max_vrp

# Create each subfigure
for subfig, data, func in zip(subfigs, datas, funcs):
    # Get data
    variances = np.mean(data["variances"], axis=0)
    corrcoefs = np.mean(data["corrcoefs"], axis=0)
    perc_diff = 100 * np.mean(data["perc_diff"], axis=0)

    axes = subfig.subplots(
        2, 2, gridspec_kw={"height_ratios": ratio, "width_ratios": ratio[::-1]}
    )
    h, ax = make_plot(
        subfig,
        func,
        axes,
        variances,
        corrcoefs,
        perc_diff,
        vmax=max_vrp if share_bar else None,
    )

# Create a single colorbar if wanted
if share_bar:
    # The 3rd number (length) will have to be changed if plotting more than two functions
    axins = ax.inset_axes(bounds=[-1.72, 1.07, 2.72, 0.1])
    cbar = fig.colorbar(
        h, cax=axins, orientation="horizontal", format=StrMethodFormatter("{x:>5.1f}%")
    )
    cbar.ax.tick_params(labelsize=cbar_tick_fs)
    cbar.ax.xaxis.set_ticks_position("top")
    cbar.ax.xaxis.set_label_position("top")
    cbar.set_label("Variance Percentage Reduction", labelpad=20, fontsize=cbar_label_fs)


---

# Correlation of $f/p$ and $g/p$ 

## Constants

In [ ]:
# Parameters for the heatmap
label_fs = 17
cmap = "gist_yarg"

# Parameters for the lines
lw = 2
alpha = 0.4
color = "#2e282a"

# Whether or not to fill contour lines
contour_fill = False

## Fontsizes of
# the figure titles
title_fs = 23
# the colorbar tick numbers
cbar_tick_fs = 15
# the colorbar label
cbar_label_fs = 20
# the axis label
axis_label_fs = 20
# the axis tick numbers
axis_tick_fs = 17

# Padding between axis label and axis ticks
axis_pad = 10
# Step size for the X and Y axes
tick_steps = [[0.05, 0.1], [0.5, 0.1]]


## Do Integrals

In [ ]:
funcs = [NGauss(16), NPolynomial(96)]
num_plots = len(funcs)
evals = 10_000
tot_iters = 100

cvis = []
for func in funcs:
    cvi = CVIntegrator(
        func, evals=evals, tot_iters=tot_iters, cv_iters="auto1", memory="large"
    )
    cvi.integrate(auto1_neval=50_000)
    cvis.append(cvi)
    print(f"{func.name}")
    print(f"CV iter: {cvi.cv_nitn[0]}")
    print(f"VRP: {100 * cvi.vrp:.2f}%")
    print(f"Mean:       {cvi.mean:.6f}")
    print(f"True value: {cvi.function.true_value:.6f}\n")


## Plot it

In [ ]:
# X and Y range for each plot
gauss16_range = [[0.9, 1.1], [0.7, 1.3]]
poly96_range = [[15, 17], [0.7, 1.3]]

corr_ranges = np.array([gauss16_range, poly96_range])
# Buffer to add onto the limits of the plot (to allow for nice looking tick numbers)
buff_val = 0.02

# Gather data
xs, ys, zs = [], [], []
for cvi, corr_range in zip(cvis, corr_ranges):
    # add buffer to range to fill out plots
    min_buff_range = corr_range[:, 0] - buff_val * abs(corr_range[:, 0])
    max_buff_range = corr_range[:, 1] + buff_val * abs(corr_range[:, 1])
    rnge = np.array([min_buff_range, max_buff_range]).T

    z, x, y = np.histogram2d(
        x=cvi.weight_value, y=cvi.cv_values[0], bins=100, range=rnge
    )
    # Shift values to correspond with ranges
    x_diff, y_diff = x[1] - x[0], y[1] - y[0]
    x = x[:-1] + x_diff
    y = y[:-1] + y_diff

    xs.append(x)
    ys.append(y)
    zs.append(z.T)


In [ ]:
# box_aspect=1 so the plots are square
fig, axes = plt.subplots(
    ncols=num_plots, figsize=(17, 8), subplot_kw=dict(box_aspect=1)
)

for x, y, z, ax, corr_range, func, cvi, tick_step in zip(
    xs, ys, zs, axes, corr_ranges, funcs, cvis, tick_steps
):
    x_lim = corr_range[0]
    y_lim = corr_range[1]
    x_ticks = np.arange(corr_range[0][0], corr_range[0][1], 0.1)

    # Either fill or don't fill
    if contour_fill:
        cont = ax.contourf(x, y, z, levels=100, cmap=cmap)
    else:
        cont = ax.contour(x, y, z, levels=8, linewidths=3, cmap="bone_r")  #'YlGn_r'
        # uncomment to have filling along with contour lines
        # cont = ax.contourf(x, y, z, levels=100, alpha=0.5, cmap='Greys') #'Greens_r'

    ax.set_ylabel("$g(x)/p(x)$", fontsize=axis_label_fs, labelpad=axis_pad)
    ax.set_xlabel("$f(x)/p(x)$", fontsize=axis_label_fs, labelpad=axis_pad)
    # Limits with the padding
    ax.set_ylim(
        y_lim[0] - 0.5 * buff_val * abs(y_lim[0]),
        y_lim[1] + 0.5 * buff_val * abs(y_lim[1]),
    )
    ax.set_xlim(
        x_lim[0] - 0.5 * buff_val * abs(x_lim[0]),
        x_lim[1] + 0.5 * buff_val * abs(x_lim[1]),
    )

    ax.tick_params(axis="both", which="major", labelsize=axis_tick_fs)
    ax.xaxis.set_major_locator(MultipleLocator(tick_step[0]))
    ax.yaxis.set_major_locator(MultipleLocator(tick_step[1]))

    # Add box with expected value and mean
    text = (
        f"$E[g/p]={np.mean(cvi.cv_values[0]):.5f}$"
        + f"\n$\\rho(f/p,g/p)={np.corrcoef(cvi.weight_value, cvi.cv_values[0])[0, 1]:.2f}$"
    )
    bbox = dict(boxstyle="round, pad=0.5", fc="wheat", ec="black", alpha=0.7)
    textbox = AnchoredText(
        text, frameon=False, loc=2, pad=1, prop=dict(fontsize=15, bbox=bbox)
    )
    ax.add_artist(textbox)

    # Add box with function name
    funcbox = AnchoredText(
        func.name,
        frameon=False,
        loc=4,
        prop=dict(fontsize=title_fs, fontweight="bold", color="#2b2d42"),
    )
    ax.add_artist(funcbox)

    # "perfect" values
    # ax.axhline(1, c=color, alpha=alpha, lw=lw)
    # ax.axvline(func.true_value, c=color, alpha=alpha, lw=lw)

fig.subplots_adjust(wspace=0.3)
if contour_fill:
    cbar = fig.colorbar(cont, ax=axes.ravel().tolist())
    cbar.ax.tick_params(labelsize=cbar_tick_fs)
    cbar.set_label("Counts", fontsize=cbar_label_fs)
# uncomment for colorbar with unfilled contour
# else:
#     cbar = fig.colorbar(cont, ax=axes.ravel().tolist())
#     cbar.ax.tick_params(labelsize=cbar_tick_fs)
#     cbar.set_label("Counts", fontsize=cbar_label_fs)
